# 3. Harmonize Raster Data and Create Model

# The below code does not work
I was unable to debug the below code with the amount of time left.
How I set up the function, I think it's meant for paths and not 
data arrays per the type error. I thought the If Else statements 
would get past that, and that was not the case. Something could 
be potentially wrong with the CRS of all of them, if I had more time 
I would verify the CRS is the same across the board or fix the whole 
difference between paths and data arrays. 

I will not be able to proceed with the habitat suitability model but 
will write what code I can for it.

In [1]:
# Download stored variables from previous notebooks

# Variables from study area download
# Project data directory
%store -r habitat_suitability_data_dir 
# Site gdfs
%store -r comanche_grassland_gdf pawnee_grassland_gdf 
# List of site gdfs 
%store -r chosen_grasslands_bounds_gdfs

# Variables from Polaris download
# List of processed polaris data arrays
%store -r  polaris_processed_da_list 
# Site specific polaris data arrays
%store -r polaris_comanche_processed polaris_pawnee_processed 

# Variables from SRTM download
# List of srtm slope data arrays
%store -r slope_da_list 
# Site specific srtm slope data arrays
%store -r slope_comanche slope_pawnee

# Variables from the MACA v2 download 
# Comanche
%store -r maca_comanche_2091_da maca_comanche_2000_da
# Pawnee
%store -r  maca_pawnee_2091_da maca_pawnee_2000_da
# MACA dataframe
%store -r maca_df

# Harmonize Data Layers: Ensure that each raster layer has 
# consistent spatial resolution and projection.


“””To harmonize data layers, ensure that all raster layers have the 
same spatial resolution and projection. The function `rio.reproject_match()` 
from the `rioxarray` library can be used to reproject and align one raster 
layer to match the resolution, extent, and CRS of a reference raster layer. 
Use `ds.rio.reproject_match()` to align all raster grids.””


In [2]:
# Prepare for download Part 1 of 1
## Import packages that will help with...

# Reproducible file paths
import os # Reproducible file paths
from glob import glob  # returns list of paths
import pathlib # Find the home folder

# Find files by pattern
import matplotlib.pyplot as plt # Overlay pandas (vector) and xarry (raster) plots 
import rioxarray as rxr # Work with geospatial raster data

# Work with tabular, vector, and raster data
import geopandas as gpd # work with vector data
import numpy as np # numerical computing
import pandas as pd # Group and aggregate
from rioxarray.merge import merge_arrays # Merge rasters
import xarray as xr # Adjust images

In [3]:
# Prep for harmonizing Part 1 of 2
# Create output directory for this harmonized raster
harmonized_raster_dir = os.path.join(habitat_suitability_data_dir, 'harmonized_rasters')
os.makedirs(harmonized_raster_dir, exist_ok=True) 

In [9]:
# Call the directory to verify location
harmonized_raster_dir

'/Users/briannagleason/earth-analytics/data/habitat_suitability/harmonized_rasters'

In [10]:
# Prep for harmonizing Part 2 of 2

# Comanche raster data arrays
comanche_reference_raster = slope_comanche

comanche_input_rasters = [
    polaris_comanche_processed,
    maca_comanche_2091_da,
    maca_comanche_2000_da
]

# Pawnee data arrays
pawnee_reference_raster = slope_pawnee

pawnee_input_rasters = [
    polaris_pawnee_processed,
    maca_pawnee_2091_da, 
    maca_pawnee_2000_da
    ]

In [11]:
# Harmonize raster layers by creating function
def harmonize_raster_layers(reference_raster, input_rasters, output_dir):
    """
    Harmonize raster layers to ensure consistent spatial resolution and projection.

    Args:
        reference_raster (str): Path to the reference raster file that other rasters will match.
        input_rasters (list): List of paths to the input raster files to harmonize.
        output_dir (str): Directory to save the harmonized raster files.

    Returns:
        list: List of file paths to the harmonized raster files.
    """
    
    # Load the reference raster
    if isinstance(reference_raster, str):  # If it's a path
        ref_raster = rxr.open_rasterio(reference_raster, masked=True)
    else:  # If it's a DataArray
        ref_raster = reference_raster

    # Create list to save the harmonized data arrays back to
    harmonized_rasters = []

    # Create for loop to iterate through multiple rasters
    for raster_path in input_rasters:

        # Load the input raster
        if isinstance(raster_path, str):  # If it's a path
            input_raster = rxr.open_rasterio(raster_path, masked=True)
        else:  # If it's a DataArray
            input_raster = raster_path

        # Reproject and align the input raster to match the reference raster
        harmonized_raster = input_raster.rio.reproject_match(ref_raster)

        # Save the harmonized raster to the output directory
        harmonized_output_file = os.path.join(output_dir, os.path.basename(raster_path))
        harmonized_raster.rio.to_raster(harmonized_output_file)

        # Print and append
        print(f"Harmonized raster saved to: {harmonized_output_file}")
        harmonized_rasters.append(harmonized_output_file)

    # Return the list of harmonized raster paths
    return harmonized_rasters


In [12]:
# Use harmonize_raster_layers function to get Comanche harmonized raster
comanche_harmonized_rasters = harmonize_raster_layers(comanche_reference_raster, comanche_input_rasters, harmonized_raster_dir)

# Use harmonize_raster_layers function to get Pawnee harmonized raster
pawnee_harmonized_rasters = harmonize_raster_layers(pawnee_reference_raster, pawnee_input_rasters, harmonized_raster_dir)


TypeError: expected str, bytes or os.PathLike object, not DataArray

# 4. Build a Model

## The following is psuedocode and has not been tested

I ran out of time to finish this and this is just code 
that could be a model but I haven't tried it on any of 
my variables. The last block of code I added my variables 
threshholds or optimal values but again, didn't run any of 
this. 

In [ ]:
# Fuzzy logic model
"""    
   - The `calculate_suitability_score` function uses 
   a **fuzzy Gaussian function** to assign suitability 
   scores between 0 and 1 for each raster cell.
   - Cells with values closer to the `optimal_value` 
   receive scores closer to 1, while cells farther away receive 
   lower scores.
   - The `tolerance_range` determines how quickly the score drops 
   off as values deviate from the optimal value.
"""

def calculate_suitability_score(raster, optimal_value, tolerance_range):
    """
    Calculate a fuzzy suitability score (0–1) for each raster 
    cell based on proximity to the optimal value.

    Args:
        raster (xarray.DataArray): Input raster layer.
        optimal_value (float): The optimal value for the variable.
        tolerance_range (float): The range within which values are considered suitable.

    Returns:
        xarray.DataArray: A raster of suitability scores (0–1).
    """
    # Calculate suitability scores using a fuzzy Gaussian function
    suitability = np.exp(-((raster - optimal_value) ** 2) / (2 * tolerance_range ** 2))
    return suitability 

suitability

In [ ]:
def build_habitat_suitability_model(
        input_rasters, optimal_values, tolerance_ranges, output_dir, threshold=None):
    """
    Build a habitat suitability model by combining fuzzy 
    suitability scores for each variable.

    Args:
        input_rasters (list): List of paths to input raster files 
            representing environmental variables.

        optimal_values (list): List of optimal values for each variable.

        tolerance_ranges (list): List of tolerance ranges for each variable.

        output_dir (str): Directory to save the combined suitability raster.

        threshold (float, optional): Threshold for highlighting highly 
            suitable areas (default is None).

    Returns:
        str: Path to the final combined suitability raster.
    """
    os.makedirs(output_dir, exist_ok=True)

    # Load and calculate suitability scores for each raster
    suitability_layers = []
    for raster_path, optimal_value, tolerance_range in zip(input_rasters, optimal_values, tolerance_ranges):
        raster = rxr.open_rasterio(raster_path, masked=True).squeeze()
        suitability_layer = calculate_suitability_score(raster, optimal_value, tolerance_range)
        suitability_layers.append(suitability_layer)

    # Combine suitability scores by multiplying across all layers
    combined_suitability = suitability_layers[0]
    for layer in suitability_layers[1:]:
        combined_suitability *= layer

    # Apply a threshold if provided
    if threshold is not None:
        combined_suitability = xr.where(combined_suitability >= threshold, combined_suitability, 0)

    # Save the combined suitability raster
    output_file = os.path.join(output_dir, "combined_suitability.tif")
    combined_suitability.rio.to_raster(output_file)
    print(f"Combined suitability raster saved to: {output_file}")

    return output_file

In [ ]:
# Example 

for site_name, site_gdf in {
    'comanche':comanche_grassland_gdf,'pawnee':pawnee_grassland_gdf}.items():
    if __name__ == "__main__":
        # Paths to input raster files (e.g., temperature, precipitation, soil pH)
        input_rasters = [
            "path_to_slope_raster/slope.tif",
            "path_to_precipitation_raster/precipitation.tif",
            "path_to_soil_ph_raster/soil_ph.tif"
        ]

        # Optimal values for Rocky Mountain Juniper for each variable
        # slope in degrees, precipitation in mm (228.6 - 660.4 mm), soil pH (5-8.5)
        optimal_values = [30, 440, 6.5]  # slope in degrees, precipitation in mm, soil pH

        # Tolerance ranges for each variable
        tolerance_ranges = [10, 220, 0.5]  # acceptable deviation for each variable

        # Output directory to save the combined suitability raster
        output_dir = "path_to_output_directory"

        # Optional threshold to highlight highly suitable areas (e.g., 0.8)
        threshold = 0.8

        # Build the habitat suitability model
        combined_suitability_file = build_habitat_suitability_model(
            input_rasters, optimal_values, tolerance_ranges, output_dir, threshold
        )

        print("Habitat suitability model created:", combined_suitability_file)

# 5. Plotting 

## Below Code is Psuedocode on how I would plot the 
## each site and climate scenario combo



I think I would need to separate each of the raster layers related 
to the site prior to plotting, but I don't think my psuedo code above 
included that. 

In [ ]:
# Comanche 2000-2004 scenario (historical)
comanche_combined_suitability_file.plot(
    cbar_kwargs={"label": "suitability score)"},
    robust=True,
    cmap='terrain',
)

# Overlay the boundary of the same study area
comanche_grassland_gdf.boundary.plot(ax=plt.gca(),
    color='black').set(
        title='Comanche Grassland-Historical Scenario 2000-2004 ',
        xlabel='Longitude', 
        ylabel='Latitude',
        xticks=[],
        yticks=[] 
    )
plt.show()

In [ ]:
# Comanche 2091-2095 scenario (rcp85)
comanche_combined_suitability_file.plot(
    cbar_kwargs={"label": "suitability score)"},
    robust=True,
    cmap='terrain',
)
# Overlay the boundary of the same study area
comanche_grassland_gdf.boundary.plot(ax=plt.gca(),
    color='black').set(
        title='Comanche Grassland- rcp85 Scenario 2091-2096  ',
        xlabel='Longitude', 
        ylabel='Latitude',
        xticks=[],
        yticks=[] 
    )
plt.show()

In [ ]:
# Pawnee 2000-2004 scenario (historical)
pawnee_combined_suitability_file.plot(
    cbar_kwargs={"label": "suitability score)"},
    robust=True,
    cmap='terrain',
)
# Overlay the boundary of the same study area
pawnee_grassland_gdf.boundary.plot(ax=plt.gca(),
    color='black').set(
        title='Pawnee Grassland- Historical Scenario 2000-2004 ',
        xlabel='Longitude', 
        ylabel='Latitude',
        xticks=[],
        yticks=[] 
    )
plt.show()

In [ ]:
# Pawnee 2091-2095 scenario (rcp85)
pawnee_combined_suitability_file.plot(
    cbar_kwargs={"label": "suitability score)"},
    robust=True,
    cmap='terrain',
)

# Overlay the boundary of the same study area
pawnee_grassland_gdf.boundary.plot(ax=plt.gca(),
    color='black').set(
        title='Pawnee Grassland- rcp85 Scenario 2091-2096 ',
        xlabel='Longitude', 
        ylabel='Latitude',
        xticks=[],
        yticks=[] 
    )
plt.show()

## Model Description - what I would want to go over if this 
## ended up working

For this Model a fuzzy logic model was employed. First my species of choice 
was researched, optimal values found for each variable used (e.g. soil pH, 
slope, and current climatological annual precipitation). For each digital number 
in each raster, a value from 0 to 1 was assigned for how close that grid square was to 
the optimum range (1=optimal, 0=incompatible). Raster layers were combined by 
multiplying them together. This will gave a single suitability number for each 
square. A threshold was applied to make the most suitable areas pop on my map.

Model results were further plotted to be able to compare habitat suitability in 
both sites, using two different scenarios...

Evaluation of the model: was this a suitable model, were there other possible ones 
that could have been use, what are the drawbacks of this model, what are the successes 
of it...



# Conclusion:

After plotting I would compare them for a conclusion:
1. Are the areas of suitability similar? Where? Does this 
line up with the context about the sites.
2. Are the areas of suitability growing, shrinking, 
changing location, etc.? 
3. Do the findings line up with the site context and context 
about the plant species or does it differ?
4. What could be futher elaborated on or is missing from 
this project?
5. What are the limitations of the conclusions?
